In [ ]:
from fastai.tabular.all import *
import json
import torch
from torch.autograd import Variable

In [ ]:
# new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
#                         [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
#                         columns=df1.columns)
df2 = pd.read_csv('../data/toy_dataset.csv')
# df2 = df2.append(new_rows, ignore_index=True)

splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['City','sex'],
                   cont_names = ['Number', 'Income', 'Age'],
                   y_names='Illness',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)

In [ ]:
# learn2.save("learn2")

In [ ]:
# learn2.model.embeds

In [ ]:
learn2.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.292761,0.285474,0.919667,00:17


In [ ]:
row, clas, probs = learn2.predict(df2.iloc[-1])

In [ ]:
row.show()

,City,sex,Number,Income,Age,Illness
0,Austin,Female,150000.001614,87251.00002,37.0,No


In [ ]:
clas, probs

(tensor(0), tensor([0.9314, 0.0686]))

In [ ]:
class TabTransfer:
    def __init__(self, json_path, new_learner, data_dir):
        self.data_dir = data_dir ## for storing models
        
        self.new_cat_names = new_learner.dls.cat_names
        self.new_all_classes = new_learner.dls.classes
        self.new_learner = new_learner
        
        self.json_path = json_path
        
#     def pickle_(self, )
    
    def read_json(self):
        f = open(self.json_path, 'rb')
        json_file = json.load(f)
        self.old_cat_names = json_file['categories']
        classes_dict = dict.fromkeys(self.old_cat_names)
        embed_dict = dict.fromkeys(self.old_cat_names)
        
        for i in self.old_cat_names:
            classes_dict[i] = json_file[i]['classes']
            embed_dict[i] = json_file[i]['embeddings']
        self.old_all_classes = classes_dict # contains the cat variable with all its classes
        self.old_all_embeds = embed_dict # contains the cat variable with all its embeddings
        
    def transfer(self, cat_names_to_transfer, verbose=False):
        self.transfer_list = cat_names_to_transfer
        
        for curr_cat in self.transfer_list:
            if not (curr_cat in self.old_cat_names and curr_cat in self.new_cat_names):
                continue
            old_cat_idx = self.old_cat_names.index(curr_cat)
            new_cat_idx = self.new_cat_names.index(curr_cat)
            
            
            try: assert (len(self.old_all_embeds[curr_cat][0]) == self.new_learner.model.embeds[new_cat_idx].embedding_dim)
            except: 
                print(f"Encountered an error for variable {curr_cat}: Make sure embeddings dimensions are same for {self.old_all_embeds[curr_cat]} and {self.new_learner.model.embeds[new_cat_idx]}")
                print("Moving on to other cat vars")
                continue
                
            
            old_curr_classes = self.old_all_classes[curr_cat]
            new_curr_classes = self.new_all_classes[curr_cat]
            global temp
            temp = new_curr_classes
            
            torch_embeds = torch.FloatTensor(self.old_all_embeds[curr_cat])
            print(torch_embeds)
            weights_mean = torch_embeds.mean(0)
            
            if verbose: print(f'mean is {weights_mean} for {torch_embeds}')

            for new_curr_class in new_curr_classes:
                new_curr_class_idx = new_curr_classes.o2i[new_curr_class]
                if verbose: print(f"{new_curr_class_idx}, {type(new_curr_class_idx)}")
                
                if new_curr_class in old_curr_classes:
                    old_curr_class_idx = old_curr_classes.index(new_curr_class)
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} from previous weights')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    tempwgt1 = self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]
                    tempwgt2 = torch_embeds[old_curr_class_idx, :]
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = torch_embeds[old_curr_class_idx, :].detach().clone()
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                else:
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} using mean')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = weights_mean
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                        
    def save_pickle(self, pickle_name):
        self.new_learner.export(self.data_dir +'/'+ f'{pickle_name}.pkl')
#             self.new_model = load_learner(data_dir +'/'+ '{pickle_name}.pkl')

    def load_pickle(self, pickle_name):
        return load_learner(self.data_dir +'/'+ f'{pickle_name}.pkl')
            
            

In [ ]:
tab_obj = TabTransfer('../data/test.json', learn2, '/Users/rakshithacharya/Desktop/Jupyter-Notebooks')

In [ ]:
tab_obj.read_json()

In [ ]:
tab_obj.transfer(["sex"], verbose = True)

tensor([[ 0.0078, -0.0057,  0.0079],
        [ 0.0084, -0.0047,  0.0006],
        [-0.0068, -0.0055,  0.0075]])
mean is tensor([ 0.0031, -0.0053,  0.0053]) for tensor([[ 0.0078, -0.0057,  0.0079],
        [ 0.0084, -0.0047,  0.0006],
        [-0.0068, -0.0055,  0.0075]])
0, <class 'int'>
Transferring weights for class #na#, cat sex using mean
old weight for class is tensor([ 0.0031, -0.0053,  0.0053], grad_fn=<SliceBackward>)
new weight for class is tensor([ 0.0031, -0.0053,  0.0053], grad_fn=<SliceBackward>)
1, <class 'int'>
Transferring weights for class Female, cat sex from previous weights
old weight for class is tensor([ 0.0078, -0.0057,  0.0079], grad_fn=<SliceBackward>)
new weight for class is tensor([ 0.0078, -0.0057,  0.0079], grad_fn=<SliceBackward>)
2, <class 'int'>
Transferring weights for class Male, cat sex from previous weights
old weight for class is tensor([ 0.0084, -0.0047,  0.0006], grad_fn=<SliceBackward>)
new weight for class is tensor([ 0.0084, -0.0047,  0.0006], 

In [ ]:
tab_obj.save_pickle('test_pickle')

In [ ]:
new_model = tab_obj.load_pickle('test_pickle')

In [ ]:
# tensor([0.9160, 0.0840])

In [ ]:
row_new_model, clas_new_model, probs_new_model = new_model.predict(df2.iloc[-1])

In [ ]:
row_new_model.show()

,City,sex,Number,Income,Age,Illness
0,Austin,Female,150000.001614,87251.00002,37.0,No


In [ ]:
clas_new_model, probs_new_model

(tensor(0), tensor([0.9316, 0.0684]))

##### =============================== #####

In [ ]:
learn2.fit(2)

epoch,train_loss,valid_loss,accuracy,time
0,0.289169,0.282669,0.919667,00:16
1,0.283433,0.279983,0.919667,00:16


In [ ]:
row, clas, probs = learn2.predict(df2.iloc[-1])

In [ ]:
row.show()

,City,sex,Number,Income,Age,Illness
0,Austin,Female,150000.001614,87251.00002,37.0,No


In [ ]:
clas, probs

(tensor(0), tensor([0.9018, 0.0982]))

In [ ]:
learner_row, learner_clas, learner_probs = new_model.predict(df2.iloc[-1])

In [ ]:
learner_row.show()

,City,sex,Number,Income,Age,Illness
0,Austin,Female,150000.001614,87251.00002,37.0,No


In [ ]:
learner_probs

tensor([0.9316, 0.0684])